## <center>**BrainScan – Détection du Cancer Cérébral par CNN**</center>

Ce projet vise à développer un modèle de Deep Learning capable de classer les images IRM cérébrales afin de détecter le cancer du cerveau avec précision. Il combine CNN, TensorFlow/Keras et OpenCV pour offrir une solution rapide, fiable et utile à l’aide au diagnostic médical.

<br>

### **Chargement et Prétraitement du Dataset :**

#### **1. Importer les bibliothèques nécessaires :**

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt